Mapa Coroplético: Mostrar en un mapa coroplético (choropleth map) la cantidad de ofertas de empleo por ciudad/comunidad/provincia. Repetir el mismo mapa pero mostrando la media de salario, segmentado por puesto de trabajo.

In [2]:
#cantidad de ofertas por ciudad/provincia.
SELECT 
    u.ubicacion,
    COUNT(o.id_urls) AS cantidad_ofertas
FROM ofertas o
JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
GROUP BY u.ubicacion
ORDER BY cantidad_ofertas DESC;

In [ ]:
#media salarial por función en cada ciudad/provincia.
SELECT 
    u.ubicacion,
    o.funcion,
    AVG((o.salario_min + o.salario_max)/2) AS salario_promedio
FROM ofertas o
JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
WHERE o.salario_min IS NOT NULL AND o.salario_max IS NOT NULL
GROUP BY u.ubicacion, o.funcion
ORDER BY salario_promedio DESC;

Outliers Salarios: Mostrar los valores de salarios usando un histograma, diferenciando los valores atípicos. Para calcular los outliers se puede usar el Z-Score, Tukey’s Fence o DBSCAN.

In [ ]:
#obtener los salarios medios de las ofertas
SELECT 
    (salario_min + salario_max) / 2 AS salario_medio
FROM ofertas
WHERE salario_min IS NOT NULL AND salario_max IS NOT NULL;

In [ ]:
# detectar outliers con Z-Score directamente en SQL (mejor en phyton)

In [ ]:
#Obtener la media y desviación estándar de salarios
SELECT 
    AVG((salario_min + salario_max)/2) AS media_salario,
    STDDEV((salario_min + salario_max)/2) AS std_salario
FROM ofertas
WHERE salario_min IS NOT NULL AND salario_max IS NOT NULL;

In [ ]:
#usar esos valores para detectar outliers (Z-Score > 3 o < -3), pero esto se hace más fácilmente y dinámicamente en Python
from scipy import stats
import pandas as pd

# df contiene la columna 'salario_medio' con los datos que sacaste de SQL
df['z_score'] = stats.zscore(df['salario_medio'])
df['outlier'] = df['z_score'].abs() > 3

Relación Salario/Ubicación: Mostrar la relación entre el salario y la ubicación de la oferta de empleo.

In [ ]:
#Salario medio por oferta + ubicación
SELECT 
    u.ubicacion,
    (o.salario_min + o.salario_max) / 2 AS salario_medio
FROM ofertas o
JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
WHERE o.salario_min IS NOT NULL AND o.salario_max IS NOT NULL;

In [ ]:
#salario promedio por ubicación (no por oferta). útil si la gráfico es más comparativo que dispersivo
SELECT 
    u.ubicacion,
    AVG((o.salario_min + o.salario_max) / 2) AS salario_promedio
FROM ofertas o
JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
WHERE o.salario_min IS NOT NULL AND o.salario_max IS NOT NULL
GROUP BY u.ubicacion
ORDER BY salario_promedio DESC;

Relación Salario/Experiencia: Mostrar si existe una relación entre el salario y la
experiencia laboral para los diferentes puestos de trabajo.

In [4]:
#Salario medio, experiencia y función (puesto)
SELECT 
    o.funcion,
    o.experiencia,
    (o.salario_min + o.salario_max) / 2 AS salario_medio
FROM ofertas o
WHERE o.salario_min IS NOT NULL 
  AND o.salario_max IS NOT NULL
  AND o.experiencia IS NOT NULL;

In [ ]:
#Opcional: agrupar el promedio por experiencia y función
SELECT 
    o.funcion,
    o.experiencia,
    AVG((o.salario_min + o.salario_max) / 2) AS salario_promedio
FROM ofertas o
WHERE o.salario_min IS NOT NULL 
  AND o.salario_max IS NOT NULL
  AND o.experiencia IS NOT NULL
GROUP BY o.funcion, o.experiencia
ORDER BY o.funcion, o.experiencia;

Habilidades Técnicas: Mostrar las habilidades técnicas y tecnologías más solicitadas para cada puesto de trabajo

In [ ]:
#Skills más solicitadas por puesto
SELECT 
  o.funcion,
  s.skills        AS skill,
  COUNT(*)        AS veces
FROM ofertas o
JOIN oferta_skill os  ON o.id_urls    = os.id_urls
JOIN skills        s  ON os.id_skills = s.id_skills
GROUP BY o.funcion, s.skills
ORDER BY o.funcion, veces DESC;

In [ ]:
#Si sólo queremos, por cada funcion, los 5 skills más frecuentes
SELECT funcion, skill, veces
FROM (
  SELECT 
    o.funcion,
    s.skills   AS skill,
    COUNT(*)   AS veces,
    ROW_NUMBER() OVER (
      PARTITION BY o.funcion 
      ORDER BY COUNT(*) DESC
    ) AS rn
  FROM ofertas o
  JOIN oferta_skill os  ON o.id_urls    = os.id_urls
  JOIN skills        s  ON os.id_skills = s.id_skills
  GROUP BY o.funcion, s.skills
) t
WHERE t.rn <= 5
ORDER BY funcion, veces DESC;

In [ ]:
#Tecnologías más solicitadas por puesto
SELECT 
  o.funcion,
  t.tecnologias_aptitudes AS tecnologia,
  COUNT(*)                AS veces
FROM ofertas o
JOIN oferta_tecnologias ot ON o.id_urls        = ot.id_urls
JOIN tecnologias_aptitudes t ON ot.id_tecnologias = t.id_tecnologias
GROUP BY o.funcion, t.tecnologias_aptitudes
ORDER BY o.funcion, veces DESC;

In [ ]:
#para quedarnos con las 5 tecnologías top por función
SELECT funcion, tecnologia, veces
FROM (
  SELECT 
    o.funcion,
    t.tecnologias_aptitudes AS tecnologia,
    COUNT(*)                AS veces,
    ROW_NUMBER() OVER (
      PARTITION BY o.funcion 
      ORDER BY COUNT(*) DESC
    ) AS rn
  FROM ofertas o
  JOIN oferta_tecnologias ot ON o.id_urls        = ot.id_urls
  JOIN tecnologias_aptitudes t ON ot.id_tecnologias = t.id_tecnologias
  GROUP BY o.funcion, t.tecnologias_aptitudes
) t
WHERE t.rn <= 5
ORDER BY funcion, veces DESC;

In [ ]:
#############otra manera

In [ ]:
#Conteo de Skills por Función
SELECT
    o.funcion,
    s.skill,
    COUNT(*) AS solicitudes_skill
FROM ofertas o
JOIN oferta_skill os
  ON o.id_urls = os.id_urls
JOIN skills s
  ON os.skill_id = s.skill_id
GROUP BY
    o.funcion,
    s.skill
ORDER BY
    o.funcion,
    solicitudes_skill DESC;

In [ ]:
#Conteo de Tecnologías por Función
SELECT
    o.funcion,
    t.tecnologia,
    COUNT(*) AS solicitudes_tecnologia
FROM ofertas o
JOIN oferta_tecnologia ot
  ON o.id_urls = ot.id_urls
JOIN tecnologia t
  ON ot.tecnologia_id = t.tecnologia_id
GROUP BY
    o.funcion,
    t.tecnologia
ORDER BY
    o.funcion,
    solicitudes_tecnologia DESC;

In [ ]:
#(Opcional) Top N Skills / Tecnologías por Función
#skills
WITH skill_counts AS (
  SELECT
    o.funcion,
    s.skill,
    COUNT(*) AS total
  FROM ofertas o
  JOIN oferta_skill os ON o.id_urls = os.id_urls
  JOIN skills s      ON os.skill_id = s.skill_id
  GROUP BY
    o.funcion,
    s.skill
)
SELECT
  funcion,
  skill,
  total
FROM (
  SELECT
    sc.*,
    ROW_NUMBER() OVER (
      PARTITION BY funcion
      ORDER BY total DESC
    ) AS rn
  FROM skill_counts sc
) ranked
WHERE rn <= 5
ORDER BY funcion, total DESC;

#tecnologias
WITH tech_counts AS (
  SELECT
    o.funcion,
    t.tecnologia,
    COUNT(*) AS total
  FROM ofertas o
  JOIN oferta_tecnologia ot ON o.id_urls = ot.id_urls
  JOIN tecnologia t        ON ot.tecnologia_id = t.tecnologia_id
  GROUP BY
    o.funcion,
    t.tecnologia
)
SELECT
  funcion,
  tecnologia,
  total
FROM (
  SELECT
    tc.*,
    ROW_NUMBER() OVER (
      PARTITION BY funcion
      ORDER BY total DESC
    ) AS rn
  FROM tech_counts tc
) ranked
WHERE rn <= 5
ORDER BY funcion, total DESC;